In [2]:
import rdkit
from rdkit import Chem
from rdkit.Chem import rdmolops

In [ ]:
from naglmbis.models import load_charge_model
# load two pre-trained charge models
gas_model = load_charge_model("nagl-gas-charge-dipole-esp-wb-default")
water_model = load_charge_model("nagl-water-charge-dipole-esp-wb-default")


In [4]:
from openff.toolkit.topology import Molecule

# create ethanol
methanol = Molecule.from_smiles("CO")
# predict the charges (in e) and atomic volumes in (bohr ^3)
gas_charges = gas_model.compute_properties(methanol.to_rdkit())["mbis-charges"]
water_charges = water_model.compute_properties(methanol.to_rdkit())["mbis-charges"]
print(gas_charges)
print(water_charges)

tensor([[-0.0773],
        [-0.5375],
        [ 0.0800],
        [ 0.0800],
        [ 0.0800],
        [ 0.3747]], grad_fn=<CatBackward0>)
tensor([[-0.0825],
        [-0.6246],
        [ 0.0904],
        [ 0.0904],
        [ 0.0904],
        [ 0.4359]], grad_fn=<CatBackward0>)


In [5]:
from naglmbis.models.base_model import ComputePartialPolarised

polarised_model = ComputePartialPolarised(
    model_gas=gas_model,
    model_water=water_model,
    alpha = 0.5
)

polarised_model.compute_polarised_charges(methanol.to_rdkit())

tensor([[-0.0799],
        [-0.5810],
        [ 0.0852],
        [ 0.0852],
        [ 0.0852],
        [ 0.4053]], grad_fn=<AddBackward0>)